<a href="https://colab.research.google.com/github/raja-jamwal/blog-building-agentic-architectures/blob/main/langchain/Part_5_Human_in_the_Loop%2C_RAG%2C_and_Inter_Agent_Communication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## Human-in-the-Loop, RAG, and Inter-Agent Communication

We have built robust, goal-oriented agents. But in the real world, we cannot always trust an AI to act alone (especially with sensitive data), nor does the AI know everything.

Today, we focus on **Trust** and **Connection**.

We will cover:
1.  **Human-in-the-Loop (HITL):** Pausing the AI to get human approval before taking action.
2.  **Knowledge Retrieval (RAG):** Giving the AI access to private data (PDFs, Docs) so it doesn't hallucinate.
3.  **Inter-Agent Communication (A2A):** How one agent can "hire" another agent to do a task.

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o-mini)
*   **Vector Store** (In-Memory for demo)


### The n8n Connection
*   **HITL** = The **Wait Node**. You can configure it to "Wait for Webhook" (e.g., a button click in an email) before the workflow continues.
*   **RAG** = The **Vector Store** nodes (Pinecone, Qdrant, Supabase). You can drag a "Retrieve from Vector Store" node to inject context into your AI Agent.
*   **A2A** = **Webhooks**. Agent A uses an HTTP Request node to send data to Agent B's Webhook URL.

In [1]:
# @title 1. Install Dependencies
# We need langchain-community and a vector store library for RAG
!pip install -qU langchain langchain-openai langchain-community faiss-cpu

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings()

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 13: Human-in-the-Loop (HITL)

**The Problem:** You want an agent to draft emails to clients, but you are terrified it might say something offensive or incorrect. You can't let it run fully autonomous.

**The Solution:** **The Approval Gate**. The agent does the heavy lifting (drafting), but pauses execution to ask a human for a "Go/No-Go" decision.

**The Scenario:** An agent drafts a refund email. We (the human) must approve it before it is "sent."

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Step 1: The Drafter ---
draft_prompt = ChatPromptTemplate.from_template(
    "Draft a polite email to a customer named {name} denying their refund request because it has been over 30 days. Keep it under 50 words."
)
chain = draft_prompt | llm | StrOutputParser()

# --- Execution with Human Intervention ---
customer = "Alice"

print("🤖 Agent is working...")
draft_email = chain.invoke({"name": customer})

print(f"\n--- 📝 DRAFT GENERATED FOR {customer.upper()} ---")
print(draft_email)
print("------------------------------------------")

# --- The "Loop" (Python Input) ---
# In n8n, this would be a 'Wait' node sending you a Slack message with buttons.
user_approval = input("Do you approve this email? (yes/no): ")

if user_approval.lower() == "yes":
    print("\n✅ Email SENT successfully.")
    # Code to send email would go here
else:
    print("\n🛑 Email ABORTED. Please refine the prompt.")
    feedback = input("Enter feedback for the agent: ")
    # In a real system, you would loop back to the Drafter with this feedback.
    print(f"Feedback '{feedback}' logged for next iteration.")

🤖 Agent is working...

--- 📝 DRAFT GENERATED FOR ALICE ---
Subject: Refund Request Update

Dear Alice,

Thank you for reaching out. Unfortunately, we cannot process your refund request as it has been over 30 days since your purchase. We appreciate your understanding and are here to assist you with any other questions.

Best regards,  
[Your Name]  
[Your Company]  
------------------------------------------
Do you approve this email? (yes/no): yes

✅ Email SENT successfully.


## Pattern 14: Knowledge Retrieval (RAG)

**The Problem:** LLMs only know what they were trained on. They don't know your company's internal HR policy or your personal notes.

**The Solution:** **RAG (Retrieval-Augmented Generation)**.
1.  **Ingest:** Break your documents into chunks and save them in a Vector Database.
2.  **Retrieve:** When a user asks a question, find the most relevant chunks.
3.  **Generate:** Send the chunks + the question to the LLM.

**The Scenario:** We will create a mini "Company Policy" database and ask the agent questions about it.

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

# --- 1. The Knowledge Base (Simulated) ---
documents = [
    "Employees are allowed 20 days of remote work per year.",
    "Expense reports must be submitted by the 5th of each month.",
    "The office kitchen is closed for cleaning on Fridays.",
    "The CEO's name is Raja."
]

# --- 2. Ingest (Create Vector Store) ---
# We turn text into numbers (embeddings) so we can search by meaning.
vectorstore = FAISS.from_texts(documents, embedding=embeddings)
retriever = vectorstore.as_retriever()

# --- 3. The RAG Prompt ---
template = """Answer the question based ONLY on the following context:
{context}

Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(template)

# --- 4. The RAG Chain ---
# RunnablePassthrough passes the question to the retriever AND the prompt
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# --- Execution ---
print("--- Query 1: Remote Work ---")
print(rag_chain.invoke("How many days can I work from home?"))

print("\n--- Query 2: Kitchen Policy ---")
print(rag_chain.invoke("Can I use the kitchen on Friday?"))

print("\n--- Query 3: Unknown Info (Hallucination Check) ---")
# It should say it doesn't know, or answer based only on context.
print(rag_chain.invoke("What is the stock price?"))

--- Query 1: Remote Work ---
You can work from home for 20 days per year.

--- Query 2: Kitchen Policy ---
No, the office kitchen is closed for cleaning on Fridays.

--- Query 3: Unknown Info (Hallucination Check) ---
The provided context does not contain any information about the stock price.


## Pattern 15: Inter-Agent Communication (A2A)

**The Problem:** You have a "Weather Agent" built in Python and a "Travel Agent" built in n8n. How do they talk?

**The Solution:** **Standardized Protocols (Agent Cards)**. Agents should expose their capabilities via APIs (Tools). One agent can "call" another agent just like it calls a calculator.

**The Scenario:** Our Main Agent needs to get data from a "Remote Weather Bot" (simulated here as a tool).

In [4]:
from langchain_core.tools import tool

# --- The "Remote" Agent ---
# Imagine this function is actually an API call to http://weather-agent.com/api
@tool
def call_remote_weather_agent(city: str) -> str:
    """Call this to get weather info from the specialized WeatherBot."""
    print(f"📡 Connecting to Remote WeatherBot for {city}...")
    # Simulating a JSON response from a remote agent
    return f"{{'agent': 'WeatherBot v2', 'city': '{city}', 'temp': '24C', 'condition': 'Sunny'}}"

# --- The Main Agent ---
tools = [call_remote_weather_agent]
main_agent = llm.bind_tools(tools)

# --- Execution ---
query = "I am planning a trip to Tokyo. Ask the weather bot what it's like there."
response = main_agent.invoke(query)

print("\n--- Main Agent Decision ---")
print(response.tool_calls)


--- Main Agent Decision ---
[{'name': 'call_remote_weather_agent', 'args': {'city': 'Tokyo'}, 'id': 'call_ixHIc2PTs6yCxDd05QFEY8Qx', 'type': 'tool_call'}]


## Summary

Your agents are now connected to reality:

1.  **HITL:** They respect human authority.
2.  **RAG:** They know your private data.
3.  **A2A:** They can collaborate with other systems.

In **Part 6**, we will optimize. We will look at **Resource-Awareness** (saving money by switching models) and **Guardrails** (ensuring safety).